In [1]:
!pip install -v -e ..

Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-ephem-wheel-cache-1mxmdax0
Created temporary directory: /tmp/pip-req-tracker-1i4a88qq
Initialized build tracking at /tmp/pip-req-tracker-1i4a88qq
Created build tracker: /tmp/pip-req-tracker-1i4a88qq
Entered build tracker: /tmp/pip-req-tracker-1i4a88qq
Created temporary directory: /tmp/pip-install-tpvpyx9f
Obtaining file:///home/riccardo/LidarObjDetection2/mmdetection3d
  Added file:///home/riccardo/LidarObjDetection2/mmdetection3d to build tracker '/tmp/pip-req-tracker-1i4a88qq'
    Running setup.py (path:/home/riccardo/LidarObjDetection2/mmdetection3d/setup.py) egg_info for package from file:///home/riccardo/LidarObjDetection2/mmdetection3d
    Running command python setup.py egg_info
    running egg_info
    writing manifest file 'mmdet3d.egg-info/SOURCES.txt'
  Source in /home/riccardo/LidarObjDetection2/mmdetection3d has version 1.4.0, which satisfies requirement mmdet3d==1.4.0 from file:///home/

In [1]:
import sys
sys.path.append("/home/riccardo/LidarObjDetection2/mmdetection3d")
from mmdet3d.apis import LidarDet3DInferencer
import os
import torch
import os
import numpy as np
import time
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
import onnxruntime as ort
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

In [12]:
# initialize inferencer 
# execute >> pip install -v -e . in mmdetection3d folder 
# CHANGE THE center_mode in loca_visualizer accordingly (KITTI -> center_mode = 'lidar_bottom', CUSTOM -> center_mode = whatever)

# inferencer = LidarDet3DInferencer('pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car')
# inferencer = LidarDet3DInferencer('pointpillars_donaset_container-car')
# inferencer = LidarDet3DInferencer('pointpillars_kittilidar-car') 
inferencer = LidarDet3DInferencer("pointpillars_hv_secfpn_8xb6-160e_lidar_vegas_iris-3d-car",scope="mmdet3d")

Loads checkpoint by local backend from path: /home/riccardo/LidarObjDetection2/mmdetection3d/work_dirs/pointpillars_hv_secfpn_8xb6-160e_lidar_vegas_iris-3d-car/iter_4750.pth


# Prediction Tests

In [3]:
import os

def copy_bin_files(source_txt, bin_folder_path,destination_folder):
  """
  Reads a list of .bin files from a text file, copies them to a separate folder,
  and handles potential errors.

  Args:
      source_txt (str): Path to the text file containing the list of .bin files.
      destination_folder (str): Path to the destination folder where the files will be copied.
  """

  # Check if source text file exists
  if not os.path.exists(source_txt):
    print(f"Error: Source text file '{source_txt}' does not exist.")
    return

  # Check if destination folder exists, create it if not
  if not os.path.exists(destination_folder):
    try:
      os.makedirs(destination_folder)
      print(f"Destination folder '{destination_folder}' created.")
    except OSError as e:
      print(f"Error creating destination folder: {e}")
      return

  # Open the source text file
  try:
    with open(source_txt, 'r') as file:
      lines = file.readlines()
  except OSError as e:
    print(f"Error opening source text file: {e}")
    return

  # Process each line in the text file
  for line in lines:
    # Remove leading/trailing whitespace and newline character
    filename = line.strip() +".bin"

    # Check if the filename ends with .bin
    if not filename.endswith('.bin'):
      print(f"Warning: Skipping non-binary file '{filename}'.")
      continue

    # Construct source and destination paths
    source_path = os.path.join(bin_folder_path, filename)
    dest_path = os.path.join(destination_folder, filename)

    # Check if source file exists
    if not os.path.exists(source_path):
      print(f"Warning: Source file '{source_path}' does not exist. Skipping...")
      continue

    # Copy the file
    try:
      with open(source_path, 'rb') as source, open(dest_path, 'wb') as destination:
        for chunk in iter(lambda: source.read(1024), b''):
          destination.write(chunk)
      print(f"Copied '{filename}' to '{destination_folder}'.")
    except OSError as e:
      print(f"Error copying file '{filename}': {e}")

# Example usage
source_txt_path = "../data/Dataset_Lidar_20240111_vegas_ontrack_01/ImageSets/val.txt"  # Replace with your actual path
destination_folder_path = "../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation"  # Replace with your actual path
bin_folder_path = "../data/Dataset_Lidar_20240111_vegas_ontrack_01/training/velodyne"
copy_bin_files(source_txt_path, bin_folder_path, destination_folder_path)


Copied '000298.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000091.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000071.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000290.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000330.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000075.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000239.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000321.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000094.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000229.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000299.bin' to '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'.
Copied '000029.bin' to '../data/Dataset_Lidar_20240111_vegas_ontr

In [15]:
# inference
pcl = '../data/Dataset_Lidar_20240111_vegas_ontrack_01/validation'
# pcl = './data/falcon/falcon1.bin'

inputs = dict(points=pcl)
inf_res = inferencer(inputs, show=True,pred_score_thr=0.4, batch_size=4)

[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed becaus

SystemExit: 0

In [9]:
inf_res

{'predictions': [{'labels_3d': [0, 0, 0, 0, 0, 0, 0, 0],
   'scores_3d': [0.8062891960144043,
    0.6793107390403748,
    0.3997745215892792,
    0.38974761962890625,
    0.2673940658569336,
    0.2191590666770935,
    0.21515747904777527,
    0.21306708455085754],
   'bboxes_3d': [[17.653263092041016,
     8.850078582763672,
     -1.2027784585952759,
     4.044853687286377,
     1.624327301979065,
     1.6509610414505005,
     4.627403259277344],
    [16.612812042236328,
     14.175690650939941,
     -1.578694224357605,
     4.175493240356445,
     1.5308202505111694,
     1.666374683380127,
     4.270689487457275],
    [11.933428764343262,
     7.8263258934021,
     -1.523880958557129,
     4.062235355377197,
     1.620255470275879,
     1.625319004058838,
     3.858912467956543],
    [38.89756393432617,
     21.367263793945312,
     -1.1178722381591797,
     4.346164226531982,
     1.6066919565200806,
     1.6587601900100708,
     1.7145425081253052],
    [22.893882751464844,
     3

In [12]:
import time
start = time.time()
inferencer(inputs, show=False,pred_score_thr=0.3,batch_size=1)
end = time.time()
print("Time Taken {} ms".format((end-start)*1000))

Time Taken 93.08862686157227 ms


In [13]:
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                 LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs
import os 

def prepare_input(inputs,batch_size=1):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [14]:
prep_input = prepare_input({"points": pcl}, batch_size=1)

Batch Size!


In [15]:
model = inferencer.model


In [16]:
import time 
import torch

torch.manual_seed(42)
s = time.time()

prep_input = prepare_input({"points": pcl}, batch_size=1) # Preprocess
res = model(prep_input["inputs"],prep_input["data_samples"], mode="tensor") # Forward
preds  = model.bbox_head.predict_by_feat(*res) # Postprocess

print("Time: {} ms".format((time.time()-s)*1000))
print(preds)

Batch Size!
Time: 65.04678726196289 ms
[<InstanceData(

    META INFORMATION

    DATA FIELDS
    labels_3d: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
    scores_3d: tensor([0.8063, 0.6793, 0.3998, 0.3897, 0.2674, 0.2192, 0.2152, 0.2131],
               device='cuda:0')
    bboxes_3d: LiDARInstance3DBoxes(
            tensor([[17.6533,  8.8501, -1.2028,  4.0449,  1.6243,  1.6510,  4.6274],
                [16.6128, 14.1757, -1.5787,  4.1755,  1.5308,  1.6664,  4.2707],
                [11.9334,  7.8263, -1.5239,  4.0622,  1.6203,  1.6253,  3.8589],
                [38.8976, 21.3673, -1.1179,  4.3462,  1.6067,  1.6588,  1.7145],
                [22.8939, 30.5794, -1.6539,  3.9149,  1.5518,  1.6938,  3.2750],
                [25.6248, 26.3890, -1.4225,  3.8247,  1.4972,  1.5861,  2.7169],
                [52.6878, -0.5002, -1.3759,  4.3683,  1.5813,  1.6613,  1.6600],
                [62.0500, -4.5474, -1.1973,  3.8232,  1.5388,  1.6548,  3.1900]],
               device='cuda:0')

In [17]:
inf_res = inferencer(inputs, show=False,pred_score_thr=0.3, batch_size=4)
inf_res

{'predictions': [{'labels_3d': [0, 0, 0, 0, 0, 0, 0, 0],
   'scores_3d': [0.8062891960144043,
    0.6793107390403748,
    0.3997745215892792,
    0.38974761962890625,
    0.2673940658569336,
    0.2191590666770935,
    0.21515747904777527,
    0.21306708455085754],
   'bboxes_3d': [[17.653263092041016,
     8.850078582763672,
     -1.2027784585952759,
     4.044853687286377,
     1.624327301979065,
     1.6509610414505005,
     4.627403259277344],
    [16.612812042236328,
     14.175690650939941,
     -1.578694224357605,
     4.175493240356445,
     1.5308202505111694,
     1.666374683380127,
     4.270689487457275],
    [11.933428764343262,
     7.8263258934021,
     -1.523880958557129,
     4.062235355377197,
     1.620255470275879,
     1.625319004058838,
     3.858912467956543],
    [38.89756393432617,
     21.367263793945312,
     -1.1178722381591797,
     4.346164226531982,
     1.6066919565200806,
     1.6587601900100708,
     1.7145425081253052],
    [22.893882751464844,
     3

Convert Manually the tensor to predictions

In [18]:
from mmdet3d.models.dense_heads.anchor3d_head import Anchor3DHead

# bbox_head=dict(
#     type='Anchor3DHead',
#     num_classes=3,
#     in_channels=384,
#     feat_channels=384,
#     use_direction_classifier=True,
#     assign_per_class=True,
#     anchor_generator=dict(
#         type='AlignedAnchor3DRangeGenerator',
#         ranges=[
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -0.6, 69.12, 39.68, -0.6],
#             [0, -39.68, -1.78, 69.12, 39.68, -1.78],
#         ],
#         sizes=[[0.8, 0.6, 1.73], [1.76, 0.6, 1.73], [3.9, 1.6, 1.56]],
#         rotations=[0, 1.57],
#         reshape_out=False),
#     diff_rad_by_sin=True,
#     bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder')
# )
# model.bbox_head
head = Anchor3DHead(
    num_classes=1,
    in_channels=384,
    feat_channels=384,
    use_direction_classifier=True,
    assign_per_class=True,
    anchor_generator=dict(
                type='AlignedAnchor3DRangeGenerator',
                ranges=[[0, -39.68, -1.78, 69.12, 39.68, -1.78]],
                sizes=[[3.9, 1.6, 1.56]],
                rotations=[0, 1.57],
                reshape_out=True),
    diff_rad_by_sin=True,
    bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder'),
    test_cfg=model.test_cfg
    )
# preds  = head.predict_by_feat(*res)
preds  = model.bbox_head.predict_by_feat(*res)
preds

/home/riccardo/LidarObjDetection2/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


[<InstanceData(
 
     META INFORMATION
 
     DATA FIELDS
     labels_3d: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
     scores_3d: tensor([0.8063, 0.6793, 0.3998, 0.3897, 0.2674, 0.2192, 0.2152, 0.2131],
                device='cuda:0')
     bboxes_3d: LiDARInstance3DBoxes(
             tensor([[17.6533,  8.8501, -1.2028,  4.0449,  1.6243,  1.6510,  4.6274],
                 [16.6128, 14.1757, -1.5787,  4.1755,  1.5308,  1.6664,  4.2707],
                 [11.9334,  7.8263, -1.5239,  4.0622,  1.6203,  1.6253,  3.8589],
                 [38.8976, 21.3673, -1.1179,  4.3462,  1.6067,  1.6588,  1.7145],
                 [22.8939, 30.5794, -1.6539,  3.9149,  1.5518,  1.6938,  3.2750],
                 [25.6248, 26.3890, -1.4225,  3.8247,  1.4972,  1.5861,  2.7169],
                 [52.6878, -0.5002, -1.3759,  4.3683,  1.5813,  1.6613,  1.6600],
                 [62.0500, -4.5474, -1.1973,  3.8232,  1.5388,  1.6548,  3.1900]],
                device='cuda:0'))
 ) at 0x7d648fa82850

In [19]:
preds[0]["bboxes_3d"],pred_res[0]["bboxes_3d"],inf_res["predictions"][0]["bboxes_3d"]

NameError: name 'pred_res' is not defined

# Model Analysis

In [3]:
model = inferencer.model
pcl_path = "../data/DonaSet/training/velodyne/"
model_path = "end2end.onnx"

In [4]:
def prepare_input(inputs,batch_size=1, device="cuda",data_samples=False):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    if data_samples:
        model_inputs["data_samples"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if data_samples:
            model_inputs["data_samples"].append(input["data_samples"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    if device == "cuda":
        prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

### Random

In [76]:
import torch
import os
import time
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

pcl_path = "../data/DonaSet/training/velodyne/"
s = time.time()

inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
# Evaluate on validation data
model.eval()
with torch.no_grad():
    val_output = model(inputs,mode="tensor")
    print("Time: {} ms".format((time.time()-s)*1000))


Batch Size!
Preprocessing took 1.7364025115966797 ms
Voxel Encoder took 2.6345252990722656 ms
Middle Encoder/Imagify took 7.281780242919922 ms
Backbone + FPN took 2.638101577758789 ms
Bbox Head forward took 0.23508071899414062 ms

Time: 17.702102661132812 ms


In [9]:
import onnx

onnx_model_path = "end2end.onnx"
m = onnx.load(onnx_model_path)
opset_version = m.opset_import[0].version if m.opset_import else None

print("ONNX Model Operator Set Version:", opset_version)

ONNX Model Operator Set Version: 11


In [10]:
import onnxruntime

ort_version = onnxruntime.__version__
print("ONNX Runtime Version:", ort_version)

print( onnxruntime.get_device()  )

ONNX Runtime Version: 1.15.1
GPU


In [80]:

import onnx
import onnxruntime as ort
import numpy as np

model_path = 'end2end.onnx'
onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)

ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])


2024-02-22 12:29:06.650643080 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 12:29:06 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-22 12:29:06.650668472 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 12:29:06 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped
2024-02-22 12:29:06.729307543 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 12:29:06 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-22 12:29:06.729332463 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 12:29:06 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped


In [81]:
s0 = time.time()
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
s05 = time.time()
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}
s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
s3 = time.time()

print("Preprocessing took: {} ms".format((s05-s0)*1000))
print("Cpu conv took: {} ms".format((s1-s05)*1000))
print("Inference took: {} ms".format((s2-s1)*1000))
print("Postprocessing took: {} ms".format((s3-s2)*1000))
print("Total: {} ms".format((s3-s0)*1000))

Batch Size!
Preprocessing took 2.275705337524414 ms
Preprocessing took: 4.957437515258789 ms
Cpu conv took: 1.6481876373291016 ms
Inference took: 18684.181928634644 ms
Postprocessing took: 3.123760223388672 ms
Total: 18693.91131401062 ms


In [82]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
s05 = time.time()
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}
s1 = time.time()


Batch Size!
Preprocessing took 2.525806427001953 ms


## Inference time comparison

### Pytorch CPU

In [7]:
model.cpu()
inputs = prepare_input({"points": pcl_path}, batch_size=1, device = "cpu")["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)

print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Batch Size!
Inf: 289.40629959106445 ms
Post: 5.438566207885742 ms
Total: 294.905424118042 ms


### Pytorch GPU

In [9]:
model.cuda()
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]

s1 = time.time()
val_output = model(inputs,mode="tensor")
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*val_output)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Batch Size!
Inf: 11.049270629882812 ms
Post: 33.72335433959961 ms
Total: 44.79646682739258 ms


### TensorRT Provider (ONNX Runtime)

In [11]:
ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

2024-02-22 15:07:18.276197313 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 15:07:18 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-22 15:07:18.276223960 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 15:07:18 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped
2024-02-22 15:07:18.357865854 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 15:07:18 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-22 15:07:18.357891501 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-22 15:07:18 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped


In [15]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))


Batch Size!
Inf: 23.366451263427734 ms
Post: 4.77290153503418 ms
Total: 28.179168701171875 ms


### CUDA Provider  (ONNX Runtime)

In [16]:
ort_sess = ort.InferenceSession(model_path,providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

In [20]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Batch Size!
Inf: 36.96608543395996 ms
Post: 6.192684173583984 ms
Total: 43.22385787963867 ms


### CPU Provider  (ONNX Runtime)

In [21]:
ort_sess = ort.InferenceSession(model_path,providers=[ 'CPUExecutionProvider'])

In [25]:
inputs = prepare_input({"points": pcl_path}, batch_size=1)["inputs"]
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}

s1 = time.time()
outputs = ort_sess.run(None, onnx_inputs)
s2 = time.time()
preds  = model.bbox_head.predict_by_feat(*outputs)
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Batch Size!
Inf: 194.2272186279297 ms
Post: 48.16293716430664 ms
Total: 242.44952201843262 ms


###  TensorRT

In [26]:
from mmdeploy.apis.inference import inference_model
from mmdeploy.apis.inference import get_model

model_cfg = "../configs/pointpillars/pointpillars_hv_secfpn_8xb6-160e_donaset-3d-car.py"
deploy_cfg = "/root/workspace/mmdeploy/configs/mmdet3d/voxel-detection/voxel-detection_tensorrt_dynamic-kitti-32x4.py"
backend_files = "end2end.engine"
img = "../data/DonaSet/training/velodyne/000000.bin"
device = "cuda"

tensorrt_model = get_model(model_cfg,deploy_cfg,[backend_files],img,device)

02/22 15:10:34 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.
02/22 15:10:34 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "mmdet3d_tasks" registry tree. As a workaround, the current "mmdet3d_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.
02/22 15:10:34 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "backend_voxel_detectors" registry tree. As a workaround, the current "backend_voxel_detectors" registry in "mmdeploy" is used to build instance. This may cause unexp

/root/workspace/LidarObjDetection/mmdetection3d_dona/mmdetection3d/mmdet3d/evaluation/functional/kitti_utils/eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


In [31]:
inputs = prepare_input({"points": pcl_path}, batch_size=1,data_samples=False)["inputs"]

s1=time.time()
res = tensorrt_model(inputs, mode="tensor")
s2 =time.time()
preds  = model.bbox_head.predict_by_feat(res["cls_score"],res["bbox_pred"],res["dir_cls_pred"])
print("Inf: {} ms".format((s2-s1)*1000))
print("Post: {} ms".format((time.time()-s2)*1000))
print("Total: {} ms".format((time.time()-s1)*1000))

Batch Size!
Inf: 1.4345645904541016 ms
Post: 25.153398513793945 ms
Total: 26.613950729370117 ms
